In [9]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Add, Activation, Dense, Input, AveragePooling2D,Flatten
from keras.models import Model

In [10]:
def bottleneck_residual_block(x, kernel_size, filters, reduce=False, s=2):
    f1, f2, f3 = filters

    x_shortcut=x

    if reduce:
        x_shortcut = Conv2D(f3, (1,1), (s,s))(x_shortcut)
        x_shortcut=BatchNormalization(axis=3)(x_shortcut)

        x=Conv2D(f1,(1,1), (s,s), 'valid')(x)
        x=BatchNormalization(axis=3)(x)
        x=Activation('relu')(x)

    else:
        x=Conv2D(f1,(1,1), (1,1), 'same')(x)
        x=BatchNormalization(axis=3)(x)
        x=Activation('relu')(x)

    x= Conv2D(f2, kernel_size, (1,1), 'same')(x)
    x=BatchNormalization(axis=3)(x)
    x=Activation('relu')(x)

    x= Conv2D(f3, (1,1), (1,1), 'valid')(x)
    x=BatchNormalization(axis=3)(x)

    x=Add()([x, x_shortcut])
    x=Activation('relu')(x)

    return x


In [13]:
def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    # part 1
    X=Conv2D(64, (7,7), (2,2), name='conv1')(X_input)
    X=BatchNormalization(axis=3, name='bn_conv1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3), (2,2))(X)

    # part 2
    X=bottleneck_residual_block(X, 3, [64, 64, 256], reduce=True, s=1)
    X=bottleneck_residual_block(X, 3, [64, 64, 256])
    X=bottleneck_residual_block(X, 3, [64, 64, 256])

    # part 3
    X=bottleneck_residual_block(X, 3, [128, 128, 512], reduce=True, s=2)
    X=bottleneck_residual_block(X, 3, [128, 128, 512])
    X=bottleneck_residual_block(X, 3, [128, 128, 512])
    X=bottleneck_residual_block(X, 3, [128, 128, 512])

    # part 4
    X=bottleneck_residual_block(X, 3, [256, 256, 1024], reduce=True, s=2)
    X=bottleneck_residual_block(X, 3, [256, 256, 1024])
    X=bottleneck_residual_block(X, 3, [256, 256, 1024])
    X=bottleneck_residual_block(X, 3, [256, 256, 1024])
    X=bottleneck_residual_block(X, 3, [256, 256, 1024])
    X=bottleneck_residual_block(X, 3, [256, 256, 1024])

    # part 5
    X=bottleneck_residual_block(X, 3, [512, 512, 2048], reduce=True, s=2)
    X=bottleneck_residual_block(X, 3, [512, 512, 2048])
    X=bottleneck_residual_block(X, 3, [512, 512, 2048])

    X= AveragePooling2D((1,1))(X)

    X=Flatten()(X)
    X=Dense(classes, 'softmax', name='fc'+str(classes))(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model

In [15]:
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
import numpy as np

epochs=200
batch_size=256


reduce_lr = ReduceLROnPlateau('val_loss', np.sqrt(0.1), patience=5, min_lr = 0.5e-6)

model = ResNet50((224,224,3), 10)
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])
